In [1]:
!pip3 install sklearn
!pip3 install pandas
!pip3 install numpy

from sklearn import preprocessing
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

     |████████████████████████████████| 10.2MB 9.7MB/s eta 0:00:01    |▍                               | 112kB 5.0MB/s eta 0:00:03     |███▋                            | 1.2MB 5.0MB/s eta 0:00:02
  Using cached https://files.pythonhosted.org/packages/e7/f9/f0b53f88060247251bf481fa6ea62cd0d25bf1b11a87888e53ce5b7c8ad2/pytz-2019.3-py2.py3-none-any.whl


In [2]:
feature_list = ['cycle_norm', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10',
               's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']

In [3]:
def process_data(train_path, test_path, RUL_path):
    # read training data - It is the aircraft engine run-to-failure data.
    train_df = pd.read_csv(train_path, sep=" ", header=None)
    train_df.drop(train_df.columns[[26, 27]], axis=1, inplace=True)
    train_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                         's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                         's15', 's16', 's17', 's18', 's19', 's20', 's21']

    train_df = train_df.sort_values(['id','cycle'])
    
    train_df = group_by_settinngs(train_df, train_path)
    
    # Data Labeling - generate column RUL (Remaining Useful Life or Time to Failure)
    rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
    rul.columns = ['id', 'max']
    train_df = train_df.merge(rul, on=['id'], how='left')
    train_df['RUL'] = train_df['max'] - train_df['cycle']
    train_df.drop('max', axis=1, inplace=True)

    # MinMax normalization (from 0 to 1)
    train_df['cycle_norm'] = train_df['cycle']
    cols_normalize = train_df.columns.difference(['id','cycle','RUL','setting1', 'setting2', 'setting3'])
    min_max_scaler = preprocessing.MinMaxScaler()
    norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]),
                                 columns=cols_normalize,
                                 index=train_df.index)
    join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
    train_df = join_df.reindex(columns = train_df.columns)  
    
    ######################
    
    # read test data - It is the aircraft engine operating data without failure events recorded.
    test_df = pd.read_csv(test_path, sep=" ", header=None)
    test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
    test_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                         's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                         's15', 's16', 's17', 's18', 's19', 's20', 's21']
    
    test_df = group_by_settinngs(test_df, test_path)
    # MinMax normalization (from 0 to 1)
    test_df['cycle_norm'] = test_df['cycle']
    norm_test_df = pd.DataFrame(min_max_scaler.transform(test_df[cols_normalize]),
                                columns=cols_normalize,
                                index=test_df.index)
    test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
    test_df = test_join_df.reindex(columns = test_df.columns)
    test_df = test_df.reset_index(drop=True)

    # read ground truth data - It contains the information of true remaining cycles for each engine in the testing data.
    truth_df = pd.read_csv(RUL_path, sep=" ", header=None)
    truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)

    # generate column max for test data
    rul = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()
    rul.columns = ['id', 'max']
    truth_df.columns = ['more']
    truth_df['id'] = truth_df.index + 1
    truth_df['max'] = rul['max'] + truth_df['more']
    truth_df.drop('more', axis=1, inplace=True)

    # generate RUL for test data
    test_df = test_df.merge(truth_df, on=['id'], how='left')
    test_df['RUL'] = test_df['max'] - test_df['cycle']
    test_df.drop('max', axis=1, inplace=True)
    
    return train_df, test_df

In [4]:
def group_by_settinngs(df, path):
    if 'FD001' in path:
        df.loc[df['setting1'].between(-0.0087, 0.0087), 'setting1'] = 0.0
        df.loc[df['setting2'].between(-0.0006, 0.0006), 'setting2'] = 0.0
        
    elif 'FD003' in path:
        df.loc[df['setting1'].between(18, 21), 'setting1'] = 20.0
        df.loc[df['setting1'].between(-0.1, 0.1), 'setting1'] = 0.0
        df.loc[df['setting1'].between(8.99, 10.9), 'setting1'] = 10.0
        df.loc[df['setting1'].between(33.9, 39.9), 'setting1'] = 35.0
        df.loc[df['setting1'].between(40.9, 42.9), 'setting1'] = 42.0
        df.loc[df['setting1'].between(23.9, 26.0), 'setting1'] = 25.0
        df.loc[df['setting2'].between(0.84, 0.85), 'setting2'] = 0.84
        df.loc[df['setting2'].between(0.6, 0.65), 'setting2'] = 0.62
        df.loc[df['setting2'].between(0.69, 0.71), 'setting2'] = 0.7
        df.loc[df['setting2'].between(33.9, 35.5), 'setting2'] = 35.0
        df.loc[df['setting2'].between(0.88, 0.95), 'setting2'] = 0.9
        
    elif 'FD003' in path:
        df.loc[df['setting1'].between(-0.0086, 0.086), 'setting1'] = 0.0
        df.loc[df['setting2'].between(-0.0006, 0.0007), 'setting2'] = 0.0
        
    elif 'FD004' in path:
        df.loc[df['setting1'].between(39.9, 35.5), 'setting1'] = 35.0
        df.loc[df['setting1'].between(-0.1, 0.1), 'setting1'] = 0.0
        df.loc[df['setting1'].between(8.99, 10.9), 'setting1'] = 10.0
        df.loc[df['setting1'].between(33.9, 35.5), 'setting1'] = 35.0
        df.loc[df['setting1'].between(40.9, 42.9), 'setting1'] = 42.0
        df.loc[df['setting1'].between(23.9, 24.9), 'setting1'] = 25.0
        df.loc[df['setting2'].between(0.84, 0.85), 'setting2'] = 0.84
        df.loc[df['setting2'].between(0.6, 0.65), 'setting2'] = 0.62
        df.loc[df['setting2'].between(0.69, 0.71), 'setting2'] = 0.7
        df.loc[df['setting2'].between(33.9, 35.5), 'setting2'] = 35.0
        df.loc[df['setting2'].between(0.88, 0.95), 'setting2'] = 0.9
    
    df.groupby('setting1')
    
    return df

In [5]:
train1,test1 = process_data('../Data/CMAPSSData/Train/train_FD001.txt','../Data/CMAPSSData/Test/test_FD001.txt', '../Data/CMAPSSData/RUL/RUL_FD001.txt')

train1.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s14,s15,s16,s17,s18,s19,s20,s21,RUL,cycle_norm
0,1,1,0.0,0.0,100.0,0.0,0.183735,0.406802,0.309757,0.0,...,0.199608,0.363986,0.0,0.333333,0.0,0.0,0.713178,0.724662,191,0.00000
1,1,2,0.0,0.0,100.0,0.0,0.283133,0.453019,0.352633,0.0,...,0.162813,0.411312,0.0,0.333333,0.0,0.0,0.666667,0.731014,190,0.00277
2,1,3,0.0,0.0,100.0,0.0,0.343373,0.369523,0.370527,0.0,...,0.171793,0.357445,0.0,0.166667,0.0,0.0,0.627907,0.621375,189,0.00554
3,1,4,0.0,0.0,100.0,0.0,0.343373,0.256159,0.331195,0.0,...,0.174889,0.166603,0.0,0.333333,0.0,0.0,0.573643,0.662386,188,0.00831
4,1,5,0.0,0.0,100.0,0.0,0.349398,0.257467,0.404625,0.0,...,0.174734,0.402078,0.0,0.416667,0.0,0.0,0.589147,0.704502,187,0.01108


In [6]:
def RUL(y):
    for i in range(len(y)):
        if y[i] > 130:
            y[i] = 130

In [7]:
train1,test1 = process_data('../Data/CMAPSSData/Train/train_FD001.txt','../Data/CMAPSSData/Test/test_FD001.txt', '../Data/CMAPSSData/RUL/RUL_FD001.txt')
train2,test2 = process_data('../Data/CMAPSSData/Train/train_FD002.txt','../Data/CMAPSSData/Test/test_FD002.txt', '../Data/CMAPSSData/RUL/RUL_FD002.txt')
train3,test3 = process_data('../Data/CMAPSSData/Train/train_FD003.txt','../Data/CMAPSSData/Test/test_FD003.txt', '../Data/CMAPSSData/RUL/RUL_FD003.txt')
train4,test4 = process_data('../Data/CMAPSSData/Train/train_FD004.txt','../Data/CMAPSSData/Test/test_FD004.txt', '../Data/CMAPSSData/RUL/RUL_FD004.txt')

In [8]:
X1_train, y1_train = np.array(train1[feature_list]), np.array(train1['RUL'])
X2_train, y2_train = np.array(train2[feature_list]), np.array(train2['RUL'])
X3_train, y3_train = np.array(train3[feature_list]), np.array(train3['RUL'])
X4_train, y4_train = np.array(train4[feature_list]), np.array(train4['RUL'])

X1_test, y1_test = np.array(test1[feature_list]), np.array(test1['RUL'])
X2_test, y2_test = np.array(test2[feature_list]), np.array(test2['RUL'])
X3_test, y3_test = np.array(test3[feature_list]), np.array(test3['RUL'])
X4_test, y4_test = np.array(test4[feature_list]), np.array(test4['RUL'])

In [9]:
def Score(estiRUL, trueRUL):
    score = 0
    for i in range(len(estiRUL)):
        h = estiRUL[i] - trueRUL[i]
        if h<0:
            score += np.exp(-h/13)-1
        else:
            score += np.exp(h/10)-1
    return score

In [10]:
def RMSE(estiRUL, trueRUL): 
    return np.sqrt(((estiRUL - trueRUL) ** 2).mean())

In [11]:
def MLP_R(X_train, y_train, X_test, y_test):

    RUL(y_train)
    RUL(y_test)
    
    mlp_r = MLPRegressor(alpha=0.1, verbose=False, batch_size=10, solver="sgd", hidden_layer_sizes=(3), max_iter=10)
    mlp_r.fit(X_train, y_train)
    pred = mlp_r.predict(X_test)
    RUL(pred)

    score = Score(pred, y_test)
    rmse = RMSE(pred, y_test)

    return "Score: ", score, "RMSE: ", rmse

In [12]:
def SVR_M(X_train, y_train, X_test, y_test):
    
    RUL(y_train)
    RUL(y_test)
    
    svr = SVR(kernel='rbf', verbose=False)
    svr.fit(X_train, y_train)
    pred = svr.predict(X_test)
    RUL(pred)

    score = Score(pred, y_test)
    rmse = RMSE(pred, y_test)
    
    return "Score: ", score, "RMSE: ", rmse

In [13]:
print ("MLP_R FD001:", MLP_R(X1_train, y1_train, X1_test, y1_test))
print ("MLP_R FD002:", MLP_R(X2_train, y2_train, X2_test, y2_test))
print ("MLP_R FD003:", MLP_R(X3_train, y3_train, X3_test, y3_test))
print ("MLP_R FD004:", MLP_R(X4_train, y4_train, X4_test, y4_test))

print ("SVR FD001:", SVR_M(X1_train, y1_train, X1_test, y1_test))
print ("SVR FD002:", SVR_M(X2_train, y2_train, X2_test, y2_test))
print ("SVR FD003:", SVR_M(X3_train, y3_train, X3_test, y3_test))
print ("SVR FD004:", SVR_M(X4_train, y4_train, X4_test, y4_test))

/usr/local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLP_R FD001: ('Score: ', 117236.3728316107, 'RMSE: ', 22.47617468091365)


/usr/local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLP_R FD002: ('Score: ', 599310.9299489153, 'RMSE: ', 25.945297840334245)


/usr/local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

SVR FD002: ('Score: ', 649112.4001936652, 'RMSE: ', 24.39471923625288)


/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR FD003: ('Score: ', 162534.41416890614, 'RMSE: ', 19.160421683631867)


/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR FD004: ('Score: ', 2667841.456914427, 'RMSE: ', 26.59382489950326)
